# MNIST
### handwritten digit prediction using neural networks

The MNIST database of handwritten digits consists of float32 Numpy tensors of shapes:

- (60000,784) for training <br>
- (10000,784) for testing

The goal of the neural network is to accurately classify handwritten digits by minimizing the discrepancy between the predicted labels and the true labels. Mini-batch Stochastic Gradient Descent (SGD) is a commonly used optimization method for this purpose.

I'll test different parameters and publish the results using Weights and Biases' "Sweeps"

## Definitions

**Intermediate Dimensions:** A higher number of dimensions usually allows the network to learn more complex representations, but it can also lead to overfitting and longer training times. A smaller number might not capture the complexity of the data.

**Learning Rates:** A larger learning rate can lead to faster convergence but might overshoot the minimum, while a smaller learning rate converges more slowly and might get stuck in local minima.

**Momentum:** A higher momentum can help to converge faster and escape local minima but might lead to instability in the training process if it's too high.

By importing and using an optimizer we can tweak the learning rate and momentum parameters

```
from tensorflow.keras import optimizers
network.compile(optimizer=optimizers.RMSprop(lr=0.001, momentum=0.0),
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
```

The first layer shown below transforms the 784-element image vector to a 512 dimensional intermediate representation. We'll iterate using different intermediate dimensions.

We'll also experiment with different learning rates and momentums.

## Build the network

The network is a chain of two dense layers that apply tensor transformations. The last one "softmax," providing the classification structure for our output, i.e., a number between 0 and 9 for each digit

````
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28, )))
network.add(layers.Dense(10, activation='softmax'))

````


In [21]:
import os
import wandb

# Retrieve the W&B entity from the environment variable
wandb_entity = os.getenv('WANDB_ENTITY')

#wandb.init(project="MNIST_experiments", entity=wandb_entity)

In [3]:
import wandb
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers, optimizers

# Function to define and train the model
def train():
    # Initialize a new W&B run
    with wandb.init() as run:
        config = run.config

        # load and preprocess the data
        (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
        train_images = train_images.reshape((60000, 28 * 28)).astype('float32') / 255
        test_images = test_images.reshape((10000, 28 * 28)).astype('float32') / 255
        train_labels = to_categorical(train_labels)
        test_labels = to_categorical(test_labels)

        # build the network
        network = models.Sequential()
        network.add(layers.Dense(config.intermediate_dimension, activation='relu', input_shape=(28 * 28,)))
        network.add(layers.Dense(10, activation='softmax'))

        # compile the network
        network.compile(optimizer=optimizers.RMSprop(lr=config.learning_rate, momentum=config.momentum),
                        loss='categorical_crossentropy', 
                        metrics=['accuracy'])

        # train the network
        history = network.fit(train_images, train_labels, 
                              epochs=config.epochs,
                              batch_size=config.batch_size, 
                              validation_data=(test_images, test_labels),
                              callbacks=[wandb.keras.WandbCallback()])

        # evaluate the network on the test set
        test_loss, test_acc = network.evaluate(test_images, test_labels)
        wandb.log({"test_loss": test_loss, "test_accuracy": test_acc})

# Sweep configuration
sweep_config = {
    'method': 'grid',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'learning_rate': {
            'values': [0.001, 0.005]
        },
        'momentum': {
            'values': [0.0, 0.25]
        },
        'intermediate_dimension': {
            'values': [256, 512, 768]
        },
        'epochs': {
            'value': 5
        },
        'batch_size': {
            'value': 128
        }
    }
}

# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, project="MNIST_experiments")

# Run the sweep
wandb.agent(sweep_id, train)



2024-02-04 19:45:44.173763: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Create sweep with ID: yeagcad6
Sweep URL: http://localhost:8080/lejomarin/MNIST_experiments/sweeps/yeagcad6


wandb: Agent Starting Run: n3aswuib with config:
wandb: 	batch_size: 128
wandb: 	epochs: 5
wandb: 	intermediate_dimension: 256
wandb: 	learning_rate: 0.001
wandb: 	momentum: 0
wandb: Currently logged in as: lejomarin. Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/5
453/469 [===========================>..] - ETA: 0s - loss: 0.2965 - accuracy: 0.9172

...

INFO:tensorflow:Assets written to: /Users/alejandromarinjimenez/Documents/Computer_Science/git/MNIST_Jan_2024/wandb/run-20240204_195104-4yzp94f7/files/model-best/assets


INFO:tensorflow:Assets written to: /Users/alejandromarinjimenez/Documents/Computer_Science/git/MNIST_Jan_2024/wandb/run-20240204_195104-4yzp94f7/files/model-best/assets
wandb: Adding directory to artifact (/Users/alejandromarinjimenez/Documents/Computer_Science/git/MNIST_Jan_2024/wandb/run-20240204_195104-4yzp94f7/files/model-best)... Done. 0.0s


469/469 [==============================] - 3s 6ms/step - loss: 0.0392 - accuracy: 0.9882 - val_loss: 0.0624 - val_accuracy: 0.9804
Epoch 5/5
313/313 [==============================] - 0s 986us/step - loss: 0.0702 - accuracy: 0.9797


accuracy,▁▆▇██
epoch,▁▃▅▆█
loss,█▃▂▁▁
test_accuracy,▁
test_loss,▁
val_accuracy,▁▄▇██
val_loss,█▅▂▁▂
accuracy,0.99115
best_epoch,3
best_val_loss,0.06242
epoch,4


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


![W&B Experiment Results](assets/W&B_Chart 2_5_2024.svg)

In [23]:
import wandb

# Replace with your actual W&B entity and project name
wandb_entity = wandb_entity
project_name = "MNIST_experiments"
sweep_id = sweep_id

# Initialize the W&B API client
api = wandb.Api()

try:
    # Fetch the sweep
    sweep = api.sweep(f"{wandb_entity}/{project_name}/{sweep_id}")
    # Get all runs in the sweep
    runs = sweep.runs
    # Get the best run (assuming you want to minimize loss)
    best_run = min(runs, key=lambda r: r.summary.get("loss", float("inf")))
    print(f"Best Run ID: {best_run.id}")
    print(f"Best Hyperparameters: {best_run.config}")
except wandb.errors.CommError as e:
    print(f"Could not access sweep: {e}")
except ValueError:
    print("No runs found for this sweep.")


Best Run ID: lm2tmx1s
Best Hyperparameters: {'epochs': 5, 'momentum': 0.25, 'batch_size': 128, 'learning_rate': 0.001, 'intermediate_dimension': 768}


In [2]:
GSD = (60000/128) * 20
print("there are ", GSD, "updates for 20 epochs of 128 sample mini-batches")

there are  9375.0 updates for 20 epochs of 128 sample mini-batches


In [2]:
# evaluate on the test set
test_loss, test_acc = network.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 4ms/step - loss: 0.0952 - accuracy: 0.9817


After fine-tuning the network we're ready to test on new, unseen data:

![W&B Experiment Results](W&B_Chart_2_5_2024.jpg)

In [24]:
from IPython.display import Image